In [6]:
import os
import tqdm
import json
import pandas as pd

In [7]:
partnet_raw_dir = "/scratch/noam/data_v0"
partnet_processed_dir = "/scratch/noam/partnet"
shapetalk_csv = "/scratch/noam/control_point_e/datasets/chair/train.csv"

In [8]:
df = pd.read_csv(shapetalk_csv)
model_id_to_shapenet_uid = {}
for _, row in df.iterrows():
    model_id_to_shapenet_uid[row.source_uid.split("/")[-1]] = row.source_uid
    model_id_to_shapenet_uid[row.target_uid.split("/")[-1]] = row.target_uid

/tmp/ipykernel_460178/2472095397.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(shapetalk_csv)


In [ ]:
shapenet_uid_to_partnet_uid = {}
partnet_uids = os.listdir(partnet_raw_dir)
for partnet_uid in tqdm.tqdm(partnet_uids, total=len(partnet_uids)):
    src_dir = os.path.join(partnet_raw_dir, partnet_uid)
    with open(os.path.join(src_dir, "meta.json"), "r") as f:
        metadata = json.load(f)
    model_id = metadata["model_id"]
    if model_id not in model_id_to_shapenet_uid:
        continue
    shapenet_uid = model_id_to_shapenet_uid[model_id]
    if shapenet_uid not in shapenet_uid_to_partnet_uid:
        shapenet_uid_to_partnet_uid[shapenet_uid] = partnet_uid
    else:
        prev_src_dir = os.path.join(partnet_raw_dir, shapenet_uid_to_partnet_uid[shapenet_uid])
        with open(os.path.join(prev_src_dir, "meta.json"), "r") as f:
            prev_metadata = json.load(f)
        if int(metadata["version"]) > int(prev_metadata["version"]):
            shapenet_uid_to_partnet_uid[shapenet_uid] = partnet_uid
with open(os.path.join(partnet_processed_dir, "shapenet_uid_to_partnet_uid.json"), "w") as f:
    json.dump(shapenet_uid_to_partnet_uid, f)